In [ ]:
# %pip install hvplot
# %pip install plotly
# %pip install holoviews
# %pip install xarray
# %pip install panel

In [ ]:
import numpy as np

import datetime as dt
import panel as pn

pn.extension()
import panel as pn
import pandas as pd
import numpy as np
import random
from datetime import datetime, timedelta

import pathlib

import holoviews as hv
import matplotlib.pyplot as plt
import hvplot
import param
import hvplot.pandas


pn.extension(sizing_mode='stretch_width')

pn.extension('perspective')

In [ ]:
# Contact Project Administrator to get these
TENANT_ID = "3aa4a235-b6e2-48d5-9195-7fcf05b459b0" # AAD using @equinor.com
CLIENT_ID = "3d7a85da-05ab-4675-b5f9-0c4b96a1c84b" # CDF_ALL_USER_APPLICATION_FLOW
CDF_CLUSTER = "westeurope-1"

# -------------for DEV project-----------------
COGNITE_PROJECT = "equinor-dev"

# -------------for TEST project-----------------
# COGNITE_PROJECT = "equinor-test"

In [ ]:
from cognite.client import CogniteClient
client = CogniteClient.default_oauth_interactive(
    project=COGNITE_PROJECT,
    tenant_id=TENANT_ID,
    client_id=CLIENT_ID,
    cdf_cluster=CDF_CLUSTER,
    client_name="my-cognite-python-test", # a name to identify your session
)
from cognite.client.data_classes import LabelFilter

In [ ]:
display(f'{client.version=} / {client.config.project=}')
# print(client.iam.token.inspect())

In [ ]:
#test

In [ ]:
wellcom_dataset = "src:015:wellcom"
edm_dataset = "src:012:edm"
witsml_dataset = "src:014:witsmldata"

In [ ]:
active_wellbore = "active:wellbore"
match_witsml = "match:witsml:wellcom" # sensor data only for johan svendrup
match_edm = "match:edm:wellcom"

# defien and show datasets

In [ ]:
#list wellcom wellbores that are active in WITSML
active_wellbores = client.assets.list(data_set_external_ids = wellcom_dataset, labels=LabelFilter(contains_all=[active_wellbore]), limit=-1)
active_wellbores

In [ ]:
witsml_db  = client.time_series.list(data_set_external_ids = witsml_dataset, limit=-1)
witsml_db

## make to padnas

In [ ]:
pd_active_wellbores = active_wellbores.to_pandas(metadata_prefix=active_wellbores)
pd_active_wellbores

In [ ]:
_pd_witsml = witsml_db.to_pandas(metadata_prefix=witsml_db)
_pd_witsml

### query pandas witsml

In [ ]:
_pd_witsml.columns

In [ ]:
import json 
import pandas as pd 
# from pandas.json_normalize import json_normalize #package for flattening json in pandas df

## flatten witsml data

In [ ]:
_pd_witsml_flattened = pd.concat([_pd_witsml.drop(['metadata'], axis=1), pd.json_normalize(_pd_witsml['metadata'])], axis=1)



In [ ]:
_pd_witsml_flattened.columns

In [ ]:
_pd_witsml_flattened = _pd_witsml_flattened[['id', 'uidWell', 'uidWellbore',  'nameWellbore','external_id', 'name', 'is_string', 'unit', 'asset_id', 'is_step',
       'description', 'security_categories', 'data_set_id', 'created_time',
       'last_updated_time', 'type', 'commonData_dTimCreation',
       'commonData_dTimLastChange', 'commonData_itemState',
       'commonData_sourceName', 'creationDate', 'direction',
       'endDateTimeIndex', 'indexCurve', 'indexType', 'name', 'nameWell',
       'nullValue', 'objectGrowing', 'serviceCompany',
       'startDateTimeIndex',  'uidLog',
       'unavailable_in_source', 'witsml-server-ref', 'source',
       'curveDescription', 'maxDateTimeIndex', 'minDateTimeIndex', 'mnemAlias',
       'mnemonic', 'typeLogData', 'uid', 'unit']]
_pd_witsml_flattened

In [ ]:
_pd_witsml_flattened[_pd_witsml_flattened['description'] == 'BITDEP']['external_id'].iloc[0]

In [ ]:
_pd_witsml_flattened['nameWell'].nunique()

In [ ]:
_df_name_is_cempumppr = _pd_witsml_flattened[_pd_witsml_flattened['description'] == 'CEMPUMPPR']

In [ ]:
_df_name_is_cempumppr.sort_values(by=['startDateTimeIndex']).head(80)

In [ ]:
len(_df_name_is_cempumppr)

In [ ]:
len(_df_name_is_cempumppr['nameWellbore'].unique())

## find sensorid to base real time data on

In [ ]:
_df_name_is_bitdep = _pd_witsml_flattened[_pd_witsml_flattened['description'] == 'BITDEP']

In [ ]:
_df_name_is_bitdep[_df_name_is_bitdep['nameWellbore'] == 'NO 16/2-D-15']

## set sensorid

In [ ]:
_sel_sensorid = int(_df_name_is_bitdep[_df_name_is_bitdep['nameWellbore'] == 'NO 16/2-D-15'].id.iloc[0])
_sel_sensorid 

#### make list of all sensorids to use in panel

In [ ]:
_sensorids_list = pd.Series(_df_name_is_bitdep['nameWellbore']).unique().tolist()
# _sensorids_list

In [ ]:
sr2 = pd.Series(pd.Series(_df_name_is_bitdep['nameWellbore']).tolist(), index = _df_name_is_bitdep['id'])
d = {key: value for key, value in sr2.items()}
_sensorids_list = d
_sensorids_list


#### panel choose _sel_sensorid

In [ ]:
# indicator = pn.indicators.LoadingSpinner(value=False, size=25)
# text = pn.widgets.TextInput(value='Start')

# def update_indicator(event, clicks):
#     if not event:
#         return
    
#     indicator.value = not indicator.value
#     text.value = f"you have clicked me {clicks} times"

# pn.bind(update_indicator, test_widget, test_widget.param.clicks, watch=True)

# pn.Column(pn.Row(test_widget, text),indicator)

In [ ]:
# _sensorids_list = [{1:"fefe"}, {2 :"foo"}]

In [ ]:
sr2 = pd.Series(pd.Series(_df_name_is_bitdep['nameWellbore']).tolist(), index = _df_name_is_bitdep['id'])
d = {key: value for key, value in sr2.items()}
_sensorids_list = d
_sensorids_list



_sel_sensorid_widget = pn.widgets.Select(name='Select', options=_sensorids_list)



# def _update__sel_sensorid
# @param.depends(_sel_sensorid_widget.param.values().__getattribute__('value'))
def _update__sel_sensorid( self, value):
    _sel_sensorid = value
    return _sel_sensorid

# bind _sel_sensorid to event _sel_sensorid_widget and display together with _sel_sensorid_widget
pn.Row(_sel_sensorid_widget, _sel_sensorid_widget.param.values().__getitem__('value'), pn.bind(_update__sel_sensorid, _sel_sensorid_widget,value=_sel_sensorid_widget.param.values().__getitem__('value'), watch=True )).servable() 

# pn.Row(_sel_sensorid_widget, pn.bind(_update__sel_sensorid, _sel_sensorid_widget,value=_sel_sensorid_widget.param.values().items())).servable()

In [ ]:
_sel_sensorid =_sel_sensorid_widget.param.values().__getitem__('value')
_sel_sensorid
## TODO why is _sel_sensorid not updating automatically?

In [ ]:
_sel_sensorid

In [ ]:
_sel_sensorid = 746172675242144

## TIME SERIES QUERY based on _sel_sensorid

In [ ]:
from cognite.client.utils import MIN_TIMESTAMP_MS, MAX_TIMESTAMP_MS

In [ ]:
client.time_series.data.retrieve(id=746172675242144,
    start=MIN_TIMESTAMP_MS,
    end=MAX_TIMESTAMP_MS + 1)

In [ ]:
_df_real_time = client.time_series.data.retrieve(
    id=_sel_sensorid,
    start=MIN_TIMESTAMP_MS,
    end=MAX_TIMESTAMP_MS + 1) 
_df_real_time


## to pandas

In [ ]:
_pd_real_time = _df_real_time.to_pandas( )
# make Value columsn or easier filtering
_pd_real_time['Value'] = _pd_real_time[_pd_real_time.columns]

In [ ]:
_pd_real_time.sort_index(ascending=True)

In [ ]:
_pd_real_time

In [ ]:

_pd_real_time_plot = _pd_real_time
# da['new_index'] = str(da.index)
# da = da.drop(columns=['dw:witsml:log.1ac57fe7-4c90-4bc7-a350-8d17f077bb32.05424d07-6a7d-4ef2-acc1-2c2a6c2eb062.05424d07-6a7d-4ef2-acc1-2c2a6c2eb062.bitdep||v1'])
# da.set_index('new_index')

In [ ]:
_pd_real_time_plot = _pd_real_time_plot.drop(_pd_real_time_plot.columns[[0]], axis=1)  # _pd_witsml_flattened.columns is zero-based pd.Index
_pd_real_time_plot['_new_index'] = _pd_real_time_plot.index
_pd_real_time_plot

In [ ]:
# from bokeh.sampledata.sea_surface_temperature import sea_surface_temperature as sst

# sst.hvplot()

In [ ]:
min(_pd_real_time_plot['_new_index'])
# Timestamp('2022-01-26 07:42:42')

In [ ]:
max(_pd_real_time_plot['_new_index'])

In [ ]:
_pd_real_time_plot[pd.isnull(_pd_real_time_plot).any(axis=1)]

## plot real time data

In [ ]:
_pd_real_time_plot.hvplot(height=1000,
    width=500, title='Real Time',
    x='Value',
    y=['_new_index'], 
    ylim=(min(_pd_real_time_plot['_new_index']), max(_pd_real_time_plot['_new_index']))
).opts(invert_yaxis=True)

## plot with panel

In [ ]:
# sr2 = pd.Series(pd.Series(_pd_witsml_flattened['id']).tolist(), index = _pd_witsml_flattened['id'])
# d = {key: value for key, value in sr2.items()}
# _sensorids_list = d
_sensorids_list = pd.Series(_pd_witsml_flattened['id']).tolist()



# create the Select widget
_sel_sensorid_widget = pn.widgets.Select(name='Select', options=_sensorids_list, width=200)



# def _update__sel_sensorid
# @param.depends(_sel_sensorid_widget.param.values().__getattribute__('value'))
def _update__sel_sensorid(value):
    _sel_sensorid = value
    # do something with _sel_sensorid
    return _sel_sensorid

# bind _sel_sensorid to event _sel_sensorid_widget and display together with _sel_sensorid_widget
# pn.Row(_sel_sensorid_widget, _sel_sensorid_widget.param.values().__getitem__('value'), pn.bind(_update__sel_sensorid, _sel_sensorid_widget,value=_sel_sensorid_widget.param.values().__getitem__('value'), watch=True )).servable() 

# pn.Row(_sel_sensorid_widget, pn.bind(_update__sel_sensorid, _sel_sensorid_widget,value=_sel_sensorid_widget.param.values().items())).servable()

#### make plot

In [ ]:

# make plot of real time data
plot =_pd_real_time_plot.hvplot(height=1000,
    width=500, title='Real Time',
    x='Value',
    y='_new_index', 
    ylim=(min(_pd_real_time_plot['_new_index']), max(_pd_real_time_plot['_new_index']))
).opts(invert_yaxis=True)




#### show panel

In [ ]:

# bind the update function to the widget
_sel_sensorid_widget.param.watch(_update__sel_sensorid, 'value')

# display the widget and plot side by side
pn.Row(_sel_sensorid_widget, plot,_sel_sensorid_widget.value).servable()

## trying with plotly

In [ ]:
pn.pane.Perspective(
    _pd_real_time,columns=['Value']
).servable()